#**CÀI ĐẶT MÔI TRƯỜNG**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#**CÀI ĐẶT THƯ VIỆN**

In [2]:
!pip install lightgbm xgboost

#**KHỞI TẠO VÀ CẤU HÌNH**

In [3]:
import os
import pandas as pd
import numpy as np
import json
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import lightgbm as lgb
from datetime import datetime

# Định nghĩa đường dẫn
SUBMISSIONS_CSV = '/content/drive/MyDrive/DATA/wecode.data/annonimized.csv'
SCORES_CSV = '/content/drive/MyDrive/DATA/wecode.data/qt-public.csv'
MODEL_DIR = '/content/drive/MyDrive/DATA/SAVE_MODEL/MODEL_SCORE_PREDICTION/QUA_TRINH/model1'
RESULT_CSV = 'result_QT.csv'

# Tạo thư mục lưu mô hình
os.makedirs(MODEL_DIR, exist_ok=True)

# Kiểm tra file tồn tại
for path, name in [(SUBMISSIONS_CSV, 'submissions'), (SCORES_CSV, 'scores')]:
    if not os.path.exists(path):
        raise FileNotFoundError(f"Không tìm thấy file {name}: {path}")
    print(f"Tìm thấy file {name}: {path}")

Tìm thấy file submissions: /content/drive/MyDrive/DATA/wecode.data/annonimized.csv
Tìm thấy file scores: /content/drive/MyDrive/DATA/wecode.data/qt-public.csv


#**TẢI VÀ KIỂM TRA DỮ LIỆU**

In [4]:
# Tải file CSV
submissions = pd.read_csv(SUBMISSIONS_CSV)
scores = pd.read_csv(SCORES_CSV)

# Kiểm tra dữ liệu
if submissions.empty:
    raise ValueError("File submissions CSV rỗng.")
if scores.empty or not {'hash', 'diemqt'}.issubset(scores.columns):
    raise ValueError("File scores CSV phải có cột 'hash' và 'TH' và không được rỗng.")

print(f"Kích thước submissions: {submissions.shape}")
print(f"Kích thước scores: {scores.shape}")

Kích thước submissions: (295198, 11)
Kích thước scores: (761, 2)


#**XỬ LÝ VÀ LÀM SẠCH DỮ LIỆU**

In [5]:

# Đổi tên cột
submissions.rename(columns={
    "concat('it001',`assignment_id`)": 'assignment_id',
    "concat('it001',`problem_id`)": 'problem_id',
    "concat('it001', username)": 'hash',  # Lưu ý khoảng trắng
    'is_final': 'is_scored',
    'status': 'raw_status',
    'pre_score': 'pct_correct',
    'coefficient': 'late_coef',
    "concat('it001',`language_id`)": 'language_id',
    'created_at': 'submit_time',
    'updated_at': 'grade_time',
    'judgement': 'judgement'
}, inplace=True)

if 'hash' not in submissions.columns:
    raise KeyError(f"Không tìm thấy cột 'hash' sau khi đổi tên. Cột hiện tại: {submissions.columns.tolist()}")

print("Danh sách cột sau khi đổi tên:", submissions.columns.tolist())

# Chuyển cột thành số
for col in ['is_scored', 'late_coef']:
    submissions[col] = pd.to_numeric(submissions[col], errors='coerce').fillna(0)


# Chuẩn hóa pct_correct
submissions['pct_correct'] = pd.to_numeric(submissions['pct_correct'], errors='coerce').fillna(0) / 100

# Tạo cột code_run = (~compilation)? 1:0
submissions['code_run'] = (~submissions['raw_status'].str.contains('Compilation Error', case=False, na=False)).astype(int)

# Xử lý thời gian
for col in ['submit_time', 'grade_time']:
    submissions[col] = pd.to_datetime(
        submissions[col],
        format='%m-%d %H:%M:%S',
        errors='coerce'
    )
    # Sửa năm 1900 thành 2025
    #mask = submissions[col].dt.year == 1900
    #submissions.loc[mask, col] += pd.DateOffset(years=125)

# Loại hàng có thời gian không hợp lệ
before = len(submissions)
submissions.dropna(subset=['submit_time', 'grade_time'], inplace=True)
print(f"Đã loại {before - len(submissions)} hàng có thời gian không hợp lệ.")

# Trích xuất số test case sai
def extract_wrong_test_cases(judgement_str):
    try:
        judgement_dict = json.loads(judgement_str.replace('""', '"'))
        verdicts = judgement_dict.get('verdicts', {})
        wrong_count = verdicts.get('WRONG', 0)
        return wrong_count
    except:
        return 0

submissions['wrong_test_cases'] = submissions['judgement'].apply(extract_wrong_test_cases)

Danh sách cột sau khi đổi tên: ['assignment_id', 'problem_id', 'hash', 'is_scored', 'raw_status', 'pct_correct', 'late_coef', 'language_id', 'submit_time', 'grade_time', 'judgement']
Đã loại 3 hàng có thời gian không hợp lệ.


In [6]:
print(submissions[['pct_correct']].head(10))
print(submissions[['submit_time', 'grade_time']].head(10))

   pct_correct
0          0.0
1          0.0
2        100.0
3        100.0
4        100.0
5          0.0
6        100.0
7          0.0
8          0.0
9          0.0
          submit_time          grade_time
0 1900-10-09 08:02:04 1900-10-09 08:06:58
1 1900-10-09 08:04:41 1900-10-09 08:04:51
2 1900-10-09 08:06:49 1900-10-09 08:06:58
3 1900-10-09 08:47:52 1900-10-09 08:48:01
4 1900-10-09 09:19:35 1900-10-09 09:19:45
5 1900-10-09 09:29:03 1900-10-09 09:30:08
6 1900-10-09 09:29:58 1900-10-09 09:30:08
7 1900-10-09 09:32:17 1900-10-09 09:42:54
8 1900-10-09 09:33:36 1900-10-09 09:33:36
9 1900-10-09 09:34:05 1900-10-09 09:34:05


#**FEATURE ENGINEERING**

In [29]:
# FEATURES TỐI ỬU HÓA CHO DỰ ĐOÁN ĐIỂM QUÁ TRÌNH - PHIÊN BẢN NÂNG CAP
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

print("=== TẠO FEATURES TỐI ỬU HÓA CHO ĐIỂM QUÁ TRÌNH ===")

# BƯỚC 1: CHUẨN BỊ DỮ LIỆU VÀ THỜI GIAN
print("Chuẩn bị dữ liệu thời gian...")
submissions['submit_date'] = submissions['submit_time'].dt.date
submissions['submit_week'] = submissions['submit_time'].dt.isocalendar().week
submissions['submit_hour'] = submissions['submit_time'].dt.hour
submissions['is_weekend'] = submissions['submit_time'].dt.weekday >= 5

# Thêm thông tin về deadline và early submission
submissions = submissions.sort_values(['hash', 'assignment_id', 'submit_time'])

# NHÓM 1: ENGAGEMENT FEATURES - QUAN TRỌNG NHẤT (8 features)
print("Tạo engagement features nâng cao...")

# Features cơ bản nhưng cải tiến
engagement_stats = submissions.groupby('hash').agg({
    'assignment_id': ['count', 'nunique'],
    'problem_id': 'nunique',
    'is_scored': 'sum',
    'submit_time': ['min', 'max']
}).round(4)

engagement_stats.columns = ['total_submissions', 'unique_assignments', 'unique_problems',
                           'total_scored', 'first_submit', 'last_submit']

# Tính tỷ lệ tham gia assignment chi tiết hơn
total_assignments = submissions['assignment_id'].nunique()
engagement_stats['assignment_coverage_rate'] = (
    engagement_stats['unique_assignments'] / total_assignments
).round(4)

# Tính submission density per assignment (quan trọng!)
engagement_stats['submissions_per_assignment'] = (
    engagement_stats['total_submissions'] / engagement_stats['unique_assignments']
).round(4)

# Early submission rate - thể hiện tính chủ động
submissions['days_from_first'] = (submissions['submit_time'] -
                                submissions.groupby('assignment_id')['submit_time'].transform('min')).dt.days
early_submissions = (submissions['days_from_first'] <= 1).groupby(submissions['hash']).mean()
engagement_stats['early_submission_rate'] = early_submissions.round(4)

# NHÓM 2: CONSISTENCY & PATTERN FEATURES - RẤT QUAN TRỌNG (6 features)
print("Tạo consistency features nâng cao...")

# Weekly consistency
weekly_activity = submissions.groupby(['hash', 'submit_week']).size().reset_index(name='weekly_submissions')
weekly_stats = weekly_activity.groupby('hash')['weekly_submissions'].agg([
    ('active_weeks', 'count'),
    ('avg_weekly_submissions', 'mean'),
    ('weekly_consistency', lambda x: 1 - x.std() / (x.mean() + 1))  # CV coefficient
]).round(4)

# Time pattern consistency
time_patterns = submissions.groupby('hash')['submit_hour'].agg([
    ('preferred_hour_consistency', lambda x: (x == x.mode().iloc[0] if len(x.mode()) > 0 else 0).mean()),
    ('weekend_work_rate', lambda x: (submissions.loc[x.index, 'is_weekend']).mean())
]).round(4)

# NHÓM 3: LEARNING PROGRESSION FEATURES - MỚI VÀ QUAN TRỌNG (5 features)
print("Tạo learning progression features...")

# Improvement over time
submissions_sorted = submissions.sort_values(['hash', 'submit_time'])
submissions_sorted['improvement_trend'] = submissions_sorted.groupby('hash')['pct_correct'].transform(
    lambda x: np.polyfit(range(len(x)), x, 1)[0] if len(x) > 1 else 0
)

learning_progression = submissions_sorted.groupby('hash').agg({
    'improvement_trend': 'first',
    'pct_correct': ['first', 'last', 'std']
}).round(4)
learning_progression.columns = ['improvement_slope', 'first_score', 'last_score', 'score_volatility']

# Learning efficiency
learning_progression['learning_efficiency'] = (
    (learning_progression['last_score'] - learning_progression['first_score']) /
    (engagement_stats['total_submissions'] + 1)
).round(4)

# NHÓM 4: PROBLEM-SOLVING BEHAVIOR - NÂNG CAP (6 features)
print("Tạo problem-solving behavior features...")

# Advanced effort metrics
submissions['attempts_per_problem'] = submissions.groupby(['hash', 'problem_id']).cumcount() + 1
submissions['is_retry'] = submissions['attempts_per_problem'] > 1

problem_solving = submissions.groupby('hash').agg({
    'attempts_per_problem': 'mean',
    'is_retry': 'mean',
    'wrong_test_cases': ['mean', 'std'],
    'code_run': 'mean',
    'pct_correct': lambda x: (x >= 0.8).mean()  # High quality completion rate
}).round(4)

problem_solving.columns = ['avg_attempts_per_problem', 'retry_rate', 'avg_wrong_cases',
                          'wrong_cases_volatility', 'code_success_rate', 'high_quality_rate']

# Strategic behavior
submissions['quick_success'] = (submissions['attempts_per_problem'] == 1) & (submissions['pct_correct'] >= 0.8)
problem_solving['strategic_success_rate'] = submissions.groupby('hash')['quick_success'].mean().round(4)

# NHÓM 5: WORKLOAD MANAGEMENT FEATURES - MỚI (4 features)
print("Tạo workload management features...")

# Session-based analysis
submissions['time_diff'] = submissions.groupby('hash')['submit_time'].diff().dt.total_seconds() / 3600
submissions['is_same_session'] = submissions['time_diff'] <= 2  # trong vòng 2h

workload_mgmt = submissions.groupby('hash').agg({
    'time_diff': lambda x: x.dropna().mean(),  # Avg time between submissions
    'is_same_session': 'mean'  # Work in sessions
}).round(4)
workload_mgmt.columns = ['avg_hours_between_submissions', 'session_work_rate']

# Workload distribution
daily_submissions = submissions.groupby(['hash', 'submit_date']).size()
workload_distribution = daily_submissions.groupby('hash').agg([
    ('max_daily_submissions', 'max'),
    ('workload_concentration', lambda x: x.std() / (x.mean() + 1))
]).round(4)

# NHÓM 6: COMPOSITE INDICES - TỐI ỬU HÓA (5 features)
print("Tạo composite indices tối ưu...")

# Merge tất cả features
all_features = engagement_stats.reset_index()
all_features = all_features.merge(weekly_stats.reset_index(), on='hash', how='left')
all_features = all_features.merge(time_patterns.reset_index(), on='hash', how='left')
all_features = all_features.merge(learning_progression.reset_index(), on='hash', how='left')
all_features = all_features.merge(problem_solving.reset_index(), on='hash', how='left')
all_features = all_features.merge(workload_mgmt.reset_index(), on='hash', how='left')
all_features = all_features.merge(workload_distribution.reset_index(), on='hash', how='left')

# Composite indices được tối ưu
all_features['engagement_index'] = (
    all_features['assignment_coverage_rate'] * 0.3 +
    all_features['submissions_per_assignment'] * 0.2 +
    all_features['early_submission_rate'] * 0.25 +
    all_features['weekly_consistency'] * 0.25
).round(4)

all_features['learning_index'] = (
    all_features['improvement_slope'] * 0.3 +
    all_features['learning_efficiency'] * 0.3 +
    all_features['high_quality_rate'] * 0.4
).round(4)

all_features['persistence_index'] = (
    all_features['retry_rate'] * 0.4 +
    all_features['strategic_success_rate'] * 0.3 +
    (1 / (all_features['workload_concentration'] + 0.1)) * 0.3
).round(4)

all_features['efficiency_index'] = (
    all_features['code_success_rate'] * 0.4 +
    (1 / (all_features['avg_attempts_per_problem'] + 0.1)) * 0.3 +
    all_features['session_work_rate'] * 0.3
).round(4)

all_features['overall_process_score'] = (
    all_features['engagement_index'] * 0.35 +
    all_features['learning_index'] * 0.25 +
    all_features['persistence_index'] * 0.2 +
    all_features['efficiency_index'] * 0.2
).round(4)

# FEATURES CUỐI CÙNG - ĐƯỢC CHỌN LỌC KỸ CÀNG
final_feature_columns = [
    'hash',

    # Core Engagement (5 features)
    'total_submissions', 'unique_assignments', 'assignment_coverage_rate',
    'submissions_per_assignment', 'early_submission_rate',

    # Consistency & Pattern (4 features)
    'active_weeks', 'weekly_consistency', 'preferred_hour_consistency', 'session_work_rate',

    # Learning Progression (4 features)
    'improvement_slope', 'learning_efficiency', 'first_score', 'last_score',

    # Problem Solving (5 features)
    'avg_attempts_per_problem', 'retry_rate', 'strategic_success_rate',
    'code_success_rate', 'high_quality_rate',

    # Workload Management (3 features)
    'avg_hours_between_submissions', 'max_daily_submissions', 'workload_concentration',

    # Composite Indices (5 features)
    'engagement_index', 'learning_index', 'persistence_index',
    'efficiency_index', 'overall_process_score'
]

# Tạo DataFrame cuối cùng
optimized_features = all_features[final_feature_columns].fillna(0)

# Handle infinite values
optimized_features = optimized_features.replace([np.inf, -np.inf], 0)

print(f"\n=== KẾT QUẢ TỐI ỬU HÓA ===")
print(f"Tổng số features: {len(final_feature_columns) - 1}")
print(f"Kích thước dữ liệu: {optimized_features.shape}")

print("\n=== CÁC NHÓM FEATURES MỚI ===")
print("1. Core Engagement (5): Tham gia cơ bản + Early submission")
print("2. Consistency & Pattern (4): Weekly + Time pattern consistency")
print("3. Learning Progression (4): Improvement trend + Learning efficiency")
print("4. Problem Solving (5): Strategic behavior + Quality metrics")
print("5. Workload Management (3): Session work + Distribution")
print("6. Composite Indices (5): Tổng hợp thông minh")

print("\n=== CẢI TIẾN CHÍNH ===")
print("✓ Thêm Early submission rate (tính chủ động)")
print("✓ Thêm Weekly consistency (đều đặn theo tuần)")
print("✓ Thêm Learning progression (xu hướng cải thiện)")
print("✓ Thêm Strategic success rate (làm đúng ngay lần đầu)")
print("✓ Thêm Session work pattern (làm việc theo session)")
print("✓ Thêm Workload management (quản lý khối lượng công việc)")
print("✓ Composite indices được tối ưu hóa với trọng số hợp lý")

# Feature importance hints
print("\n=== DỰ ĐOÁN FEATURES QUAN TRỌNG NHẤT ===")
print("1. assignment_coverage_rate - Tỷ lệ assignment đã làm")
print("2. engagement_index - Chỉ số tham gia tổng hợp")
print("3. weekly_consistency - Đều đặn theo tuần")
print("4. early_submission_rate - Tỷ lệ nộp sớm")
print("5. learning_efficiency - Hiệu quả học tập")

# Export
optimized_features.to_csv('optimized_features_for_process_score.csv', index=False)
print("\n✓ Đã lưu vào 'optimized_features_for_process_score.csv'")
print("\n🚀 Features đã được tối ưu hóa toàn diện cho R² score cao hơn!")

=== TẠO FEATURES TỐI ỬU HÓA CHO ĐIỂM QUÁ TRÌNH ===
Chuẩn bị dữ liệu thời gian...
Tạo engagement features nâng cao...
Tạo consistency features nâng cao...
Tạo learning progression features...
Tạo problem-solving behavior features...
Tạo workload management features...
Tạo composite indices tối ưu...

=== KẾT QUẢ TỐI ỬU HÓA ===
Tổng số features: 26
Kích thước dữ liệu: (1489, 27)

=== CÁC NHÓM FEATURES MỚI ===
1. Core Engagement (5): Tham gia cơ bản + Early submission
2. Consistency & Pattern (4): Weekly + Time pattern consistency
3. Learning Progression (4): Improvement trend + Learning efficiency
4. Problem Solving (5): Strategic behavior + Quality metrics
5. Workload Management (3): Session work + Distribution
6. Composite Indices (5): Tổng hợp thông minh

=== CẢI TIẾN CHÍNH ===
✓ Thêm Early submission rate (tính chủ động)
✓ Thêm Weekly consistency (đều đặn theo tuần)
✓ Thêm Learning progression (xu hướng cải thiện)
✓ Thêm Strategic success rate (làm đúng ngay lần đầu)
✓ Thêm Session w

#**GỘP FEATURES VỪA TẠO**

In [30]:
# Chuyển TH thành số
scores['diemqt'] = pd.to_numeric(scores['diemqt'], errors='coerce')

# Gộp features với scores
if not optimized_features.empty and not scores.empty and 'hash' in optimized_features.columns:
    data = optimized_features.merge(scores, on='hash', how='left')
    print(f"Đã gộp features và scores. Kích thước data: {data.shape}")
    # Hiển thị tất cả các cột
    pd.set_option('display.max_columns', None)

    # In dữ liệu mẫu đầy đủ
    print("Dữ liệu mẫu:\n", data.head(10))
    print(f"Số giá trị NaN trong 'diemqt': {data['diemqt'].isnull().sum()}")
else:
    print("Cảnh báo: Không thể gộp do features hoặc scores rỗng, hoặc thiếu cột 'hash'.")
    data = pd.DataFrame(columns=optimized_features.columns.tolist() + ['diemqt'])

Đã gộp features và scores. Kích thước data: (1489, 28)
Dữ liệu mẫu:
                                        hash  total_submissions  \
0  00b6dd4fc7eb817e03708c532016ef30ce564a61                147   
1  00bef8afee8f3c595d535c9c03c490cac1a4f021                259   
2  01122b3ef7e59b84189e65985305f575d6bdf83c                195   
3  0134f9f410c65ad0e8c2254a7e9288670e02a183                100   
4  013de369c439ab0ead8aa7da64423aa395a8be39                107   
5  014c59c6433fd764a0b08de6ffeb757eaf60aa73                199   
6  0155f91fe242fe55e1af631b0f0afbe9938b185c                179   
7  016f87a7bd87f233a8633fba8a1ca7ef5c3c63c1                364   
8  017ec1bbad0d598e5a86ac3bd595b870b6456e61                218   
9  01ad03ec301749c033155fc17e657cc767d447c8                271   

   unique_assignments  assignment_coverage_rate  submissions_per_assignment  \
0                   7                    0.0345                     21.0000   
1                   9                    0.044

#**CHUẨN BỊ DỮ LIỆU HUẤN LUYỆN**

In [31]:
# Chuẩn bị tập huấn luyện/kiểm tra
if not data.empty and 'diemqt' in data.columns and pd.api.types.is_numeric_dtype(data['diemqt']):
    train = data.dropna(subset=['diemqt'])
    print(f"Kích thước dữ liệu huấn luyện: {train.shape}")

    if train.empty:
        raise ValueError("Không có hàng nào có điểm TH hợp lệ. Kiểm tra 'hash' và th-public.csv.")

    # Loại cột
    cols_to_drop = [col for col in ['hash', 'diemqt'] if col in train.columns]
    X = train.drop(cols_to_drop, axis=1)
    y = train['diemqt']

    # Chia tập huấn luyện và kiểm tra
    if X.shape[1] > 0:
        X_train, X_val, y_train, y_val = train_test_split(
            X, y, test_size=0.2, random_state=42
        )
        print(f"STập huấn luyện: X {X_train.shape}, y {y_train.shape}")
        print(f"STập kiểm tra: X {X_val.shape}, y {y_val.shape}")

        # Tạo Dataset
        train_data = lgb.Dataset(X_train, label=y_train)
        val_data = lgb.Dataset(X_val, label=y_val, reference=train_data)
    else:
        print("Cảnh báo: nKhông có tính năng để huấn luyện.")
        train_data, val_data = None, None
        X_train, X_val = pd.DataFrame(), pd.DataFrame()
        y_train, y_val = pd.Series(dtype=float), pd.Series(dtype=float)
else:
    print("Cảnh báo: Dữ Dữ liệu rỗng hoặc thiếu cột 'diemqt'/không phải số. Bỏ qua chia tập.")
    train_data, val_data = None, None
    X_train, X_val = pd.DataFrame(), pd.DataFrame()
    y_train, y_val = pd.Series(dtype=float), pd.Series(dtype=float)

Kích thước dữ liệu huấn luyện: (754, 28)
STập huấn luyện: X (603, 26), y (603,)
STập kiểm tra: X (151, 26), y (151,)


#**HUẤN LUYỆN MODEL CƠ BẢN**

In [32]:
import lightgbm as lgb
import xgboost as xgb
from sklearn.metrics import r2_score, mean_squared_error
import numpy as np
import os
import time

# Kiểm tra dữ liệu huấn luyện
if train_data is not None and X_train.shape[1] > 0:



    results = {}
    print("1. HUẤN LUYỆN LIGHTGBM")

    start_time = time.time()

    # Tham số LightGBM
    lgb_params = {
        'objective': 'regression',
        'metric': 'rmse',
        'num_leaves': 100,
        'learning_rate': 0.03,
        'feature_fraction': 0.9,
        'bagging_fraction': 0.8,
        'bagging_freq': 3,
        'verbose': -1,
        'random_state': 42
    }

    try:
        # Huấn luyện LightGBM
        model_lgb = lgb.train(
            lgb_params,
            train_data,
            num_boost_round=500,
            valid_sets=[val_data],
            callbacks=[lgb.early_stopping(stopping_rounds=100)]
        )

        # Lưu mô hình
        lgb_model_path = os.path.join(MODEL_DIR, 'lightgbm_model.txt')
        model_lgb.save_model(lgb_model_path)

        # Đánh giá
        y_pred_lgb = model_lgb.predict(X_val)
        r2_lgb = r2_score(y_val, y_pred_lgb)
        rmse_lgb = np.sqrt(mean_squared_error(y_val, y_pred_lgb))
        training_time_lgb = time.time() - start_time

        # Lưu kết quả
        results['LightGBM'] = {
            'model': model_lgb,
            'r2': r2_lgb,
            'rmse': rmse_lgb,
            'time': training_time_lgb,
            'path': lgb_model_path,
            'predictions': y_pred_lgb
        }

        print(f" LightGBM huấn luyện thành công!")
        print(f"   R² score: {r2_lgb:.4f}")
        print(f"   RMSE: {rmse_lgb:.4f}")
        print(f"   Thời gian: {training_time_lgb:.2f}s")
        print(f"   Đã lưu tại: {lgb_model_path}")

    except Exception as e:
        print(f" Lỗi LightGBM: {e}")
        results['LightGBM'] = None

    print("2. HUẤN LUYỆN XGBOOST")

    start_time = time.time()

    try:
        # Tạo DMatrix cho XGBoost
        dtrain = xgb.DMatrix(X_train, label=y_train)
        dval = xgb.DMatrix(X_val, label=y_val)

        # Tham số XGBoost
        xgb_params = {
            'objective': 'reg:squarederror',
            'eval_metric': 'rmse',
            'max_depth': 6,
            'learning_rate': 0.03,
            'subsample': 0.8,
            'colsample_bytree': 0.9,
            'random_state': 42,
            'verbosity': 0
        }

        # Huấn luyện XGBoost
        model_xgb = xgb.train(
            xgb_params,
            dtrain,
            num_boost_round=500,
            evals=[(dval, 'validation')],
            early_stopping_rounds=100,
            verbose_eval=False
        )


        # Lưu mô hình
        xgb_model_path = os.path.join(MODEL_DIR, 'xgboost_model.json')
        model_xgb.save_model(xgb_model_path)

        # Đánh giá
        y_pred_xgb = model_xgb.predict(dval)
        r2_xgb = r2_score(y_val, y_pred_xgb)
        rmse_xgb = np.sqrt(mean_squared_error(y_val, y_pred_xgb))  # Tính RMSE thủ công
        training_time_xgb = time.time() - start_time

        # Lưu kết quả
        results['XGBoost'] = {
            'model': model_xgb,
            'r2': r2_xgb,
            'rmse': rmse_xgb,
            'time': training_time_xgb,
            'path': xgb_model_path,
            'predictions': y_pred_xgb
        }

        print(f" XGBoost huấn luyện thành công!")
        print(f"   R² score: {r2_xgb:.4f}")
        print(f"   RMSE: {rmse_xgb:.4f}")
        print(f"   Thời gian: {training_time_xgb:.2f}s")
        print(f"   Đã lưu tại: {xgb_model_path}")

    except Exception as e:
        print(f" Lỗi XGBoost: {e}")
        results['XGBoost'] = None


    print("3. SO SÁNH KẾT QUẢ")


    # Tạo bảng so sánh
    print(f"{'Mô hình':<12} {'R² Score':<10} {'RMSE':<8} {'Thời gian':<10} {'Trạng thái':<12}")
    print("-" * 60)

    best_model = None
    best_r2 = -999

    for model_name, result in results.items():
        if result is not None:
            status = " Thành công"
            r2_str = f"{result['r2']:.4f}"
            rmse_str = f"{result['rmse']:.4f}"
            time_str = f"{result['time']:.2f}s"

            # Tìm mô hình tốt nhất
            if result['r2'] > best_r2:
                best_r2 = result['r2']
                best_model = model_name
        else:
            status = "Lỗi"
            r2_str = rmse_str = time_str = "N/A"

        print(f"{model_name:<12} {r2_str:<10} {rmse_str:<8} {time_str:<10} {status:<12}")

    # Thông báo mô hình tốt nhất
    if best_model:
        print(f"\n MÔ HÌNH TỐT NHẤT: {best_model} (R² = {best_r2:.4f})")
        print(f" Đường dẫn: {results[best_model]['path']}")

else:
    print(" CẢNH BÁO: Dữ liệu huấn luyện rỗng hoặc thiếu tính năng/target.")
    print("Thông tin debug:")
    print(f"- Kích thước X_train: {X_train.shape if 'X_train' in globals() else 'Chưa định nghĩa'}")
    print(f"- train_data exists: {train_data is not None if 'train_data' in globals() else 'Chưa định nghĩa'}")
    results = {'LightGBM': None, 'XGBoost': None}

1. HUẤN LUYỆN LIGHTGBM
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[111]	valid_0's rmse: 1.30355
 LightGBM huấn luyện thành công!
   R² score: 0.2823
   RMSE: 1.3035
   Thời gian: 0.25s
   Đã lưu tại: /content/drive/MyDrive/DATA/SAVE_MODEL/MODEL_SCORE_PREDICTION/QUA_TRINH/model1/lightgbm_model.txt
2. HUẤN LUYỆN XGBOOST
 XGBoost huấn luyện thành công!
   R² score: 0.2647
   RMSE: 1.3194
   Thời gian: 1.48s
   Đã lưu tại: /content/drive/MyDrive/DATA/SAVE_MODEL/MODEL_SCORE_PREDICTION/QUA_TRINH/model1/xgboost_model.json
3. SO SÁNH KẾT QUẢ
Mô hình      R² Score   RMSE     Thời gian  Trạng thái  
------------------------------------------------------------
LightGBM     0.2823     1.3035   0.25s       Thành công 
XGBoost      0.2647     1.3194   1.48s       Thành công 

 MÔ HÌNH TỐT NHẤT: LightGBM (R² = 0.2823)
 Đường dẫn: /content/drive/MyDrive/DATA/SAVE_MODEL/MODEL_SCORE_PREDICTION/QUA_TRINH/model1/lightgbm_model.txt


#**DỰ ĐOÁN VÀ XUẤT KẾT QUẢ**

In [ ]:
# Dự đoán cho tất cả hash
if model_lgb is not None and not features.empty and 'hash' in features.columns:
    X_all = features.drop('hash', axis=1)
    # Đảm bảo cột khớp với tập huấn luyện
    for c in X_train.columns.difference(X_all.columns):
        X_all[c] = 0
    X_all = X_all[X_train.columns]
    preds = model_lgb.predict(X_all)

    # Tạo dataframe kết quả
    submission = pd.DataFrame({
        'hash': features['hash'],
        'CK_pred': np.round(preds, 2)
    })
    submission_path = os.path.join(MODEL_DIR, RESULT_CSV)
    submission.to_csv(submission_path, index=False)
    print(f"Đã lưu kết quả tại: {submission_path}")

    # Tải file kết quả
    from google.colab import files
    files.download(submission_path)
else:
    print("Cảnh báo: Mô hình chưa được huấn luyện hoặc features rỗng.")
    print("model exists:", model_lgb is not None if 'model' in globals() else "Chưa định nghĩa")
    print("features exists:", not features.empty if 'features' in globals() else "Chưa định nghĩa")
    print("hash in features:", 'hash' in features.columns if 'features' in globals() and not features.empty else "Chưa định nghĩa")

#**BIỂU ĐỒ GIỮA GIÁ TRỊ DỰ ĐOÁN VÀ GIÁ TRỊ THỰC**

In [ ]:
import matplotlib.pyplot as plt

def plot_predictions(y_true, y_pred, model_name):
    plt.figure(figsize=(8, 6))
    plt.scatter(y_true, y_pred, alpha=0.5, label='Dự đoán')
    plt.plot([y_true.min(), y_true.max()], [y_true.min(), y_true.max()], 'r--', label='Đường hoàn hảo')
    plt.xlabel('Actual QT Scores')
    plt.ylabel('Predicted QT Scores')
    plt.title(f'Predicted vs Actual QT Scores ({model_name})')
    plt.legend()
    plt.grid(alpha=0.3)
    plt.show()


In [ ]:
    # Dự đoán trên tập kiểm tra với LightGBM
    y_pred_lgb = model_lgb.predict(X_val)

    # Tạo biểu đồ cho LightGBM
    plot_predictions(y_val, y_pred_lgb, model_name='LightGBM')

    # Dự đoán trên tập kiểm tra với XGBoost
    # Chuyển đổi X_val thành DMatrix
    dval = xgb.DMatrix(X_val)
    y_pred_xgb = model_xgb.predict(dval)

    # Tạo biểu đồ cho XGBoost
    plot_predictions(y_val, y_pred_xgb, model_name='XGBoost')